In [15]:
import pandas as pd
import sqlite3
import sqlalchemy
import random
from datetime import timedelta
import numpy as np

In [16]:
reactions = pd.read_csv('sampleReactions.csv', sep="\t")
hashtags = ["#stayhome", "#lockdown", "#lockdown", "#socialdistancing", "#socialdistancing", "#lockdown", "#longcovid", "#freeagain", "#rip", "#unemployed", "#homeschooling", "#homeoffice", "#rip", "#unemployed", "#homeschooling", "#homeoffice", "#workfromhome", "#workfromhome", "#stayhome", "#stayhome", "#vaccination", "#quarantine", "#flattenthecurve", "#stayhome", "#lockdown", "#longcovid"]
for (i, row) in enumerate(reactions.iterrows()):
    reactions.loc[i, 'comment'] = row[1]['comment'] + " " +random.choice(hashtags)
    
reactions['comment']


0      while most the country was all about Baby Shar...
1                What happened with Chernobyl? #stayhome
2      Surprised there wasn’t another Baby Shark spik...
3      Montana being the only state with Far Cry 5 wa...
4      Fortnite lasted longer than George Floydd, BLM...
                             ...                        
470    I remember these huge ships getting paid to ta...
471    Why does the simulation end up making the patc...
472    Notice the steady stream coming out of China. ...
473    this seems like bullshit... &#x200B; In Januar...
474    What’s brilliant about that is it works like a...
Name: comment, Length: 475, dtype: object

In [17]:
df = pd.read_json("./sampleComments.json")
df['id'] = df.index
df = df.drop(['value'], axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df['date'] = df['date'].dt.date
df['enddate'] = df['date']
df['createdAt'] = pd.to_datetime('today').normalize()
df['updatedAt'] = pd.to_datetime('today').normalize()

#df.drop(df[df.country != "Germany"].index, inplace=True)

for (i, row) in enumerate(df.iterrows()):
    reaction = reactions.sample()
    df.loc[i, 'comment'] = reaction['comment'].values[0]
    df.loc[i, 'author'] = reaction['author'].values[0]
    if (i % 3) is 0:
        newDate = row[1].date + timedelta(days=random.randint(5,20))
        df.loc[i, 'enddate'] = newDate


df

,date,comment,weight,country,id,enddate,createdAt,updatedAt,author
0,2021-01-22,Is there anything beautiful about this? The da...,4,Fiji,0,2021-02-01,2021-03-11,2021-03-11,bth807
1,2020-06-11,During all past recessions people could work u...,4,Lebanon,1,2020-06-11,2021-03-11,2021-03-11,mrgogonuts
2,2020-08-10,We don't have daylight savings here in India. ...,4,Iceland,2,2020-08-10,2021-03-11,2021-03-11,vrnbhargava
3,2020-07-26,What's even more surprising is the inflation o...,4,Lesotho,3,2020-08-02,2021-03-11,2021-03-11,opinionsareus
4,2020-09-08,Please tell me why is being billionaire bad? I...,3,Mongolia,4,2020-09-08,2021-03-11,2021-03-11,mrmatdamon
...,...,...,...,...,...,...,...,...,...
49995,2020-11-05,Anyone unhappy about these numbers probably ha...,3,El Salvador,49995,2020-11-19,2021-03-11,2021-03-11,ImViddy
49996,2020-03-16,Not sure if these low prices are good or bad. ...,4,Latvia,49996,2020-03-16,2021-03-11,2021-03-11,JM-Gurgeh
49997,2020-06-29,"Shoulda kept it going, does Obama catch up in ...",4,Iceland,49997,2020-06-29,2021-03-11,2021-03-11,Gabernasher
49998,2020-10-31,Ooh this is some fresh data. Assuming this is ...,4,Luxembourg,49998,2020-11-12,2021-03-11,2021-03-11,landonairey


In [18]:
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
df.to_sql(con=sqlite, index=False, name='memories', if_exists='replace')